In [1]:
%matplotlib widget
from spectraclass.data.base import DataManager
from spectraclass.data.base import ModeDataManager
from spectraclass.data.spatial.tile.manager import TileManager
from spectraclass.data.spatial.modes import AvirisDataManager
from spectraclass.application.controller import app, SpectraclassController
from spectraclass.model.labels import LabelsManager, lm
from spectraclass.learn.manager import ClassificationManager, cm
from spectraclass.learn.base import LearningModel
import os
from typing import List, Union, Tuple, Optional, Dict, Callable

Here we configure paths on the Jupyter server.  If these paths are not specified here then the default values,
    defined in server-side config files, for the project ("demo2") and data mode ("desis"), will be used.  You can
    choose whatever project names you want, they are used to save configurations and results for ongoing investigations.

In [2]:
dm: DataManager = DataManager.initialize( "img_mgr", 'aviris' )
location = "desktop"
if location == "adapt":
    dm.modal.cache_dir = "/adapt/nobackup/projects/ilab/cache"
    dm.modal.data_dir = "/css/above/daac.ornl.gov/daacdata/above/ABoVE_Airborne_AVIRIS_NG/data/"
elif location == "desktop":
    dm.modal.cache_dir = "/Volumes/Shared/Cache"
    dm.modal.data_dir = "/Users/tpmaxwel/Development/Data/Aviris"
else: raise Exception( f"Unknown location: {location}")
    
block_size = 200
method = "aec" # "vae"
model_dims = 32
version = "v2p9"  # "v2v2" "v2p9"
preprocess = False

dm.use_model_data = True
dm.proc_type = "cpu"
dm.modal.images_glob = f"ang*rfl/ang*_rfl_{version}/ang*_corr_{version}_img"
TileManager.block_size = block_size
TileManager.block_index = [1,5]
AvirisDataManager.version = version
dm.modal.valid_aviris_bands = [ [5,193], [214,283], [319,10000] ]
dm.modal.model_dims = model_dims
dm.modal.reduce_method = method
dm.modal.reduce_nepoch = 3
dm.modal.reduce_focus_nepoch = 10
dm.modal.reduce_niter = 1
dm.modal.reduce_focus_ratio = 10.0
dm.modal.reduce_dropout = 0.0
dm.modal.reduce_learning_rate = 1e-4
dm.modal.refresh_model = False
dm.modal.modelkey = f"b{block_size}.{method}"


Opening log file:  '/Users/tpmaxwel/.spectraclass/logging/aviris/img_mgr.log'
Using config file: '/Users/tpmaxwel/Development/Projects/spectraclass/defaults/config.py'
Using config file: '/Users/tpmaxwel/.spectraclass/config/aviris/img_mgr.py'


Here we load the project data and define a set of class names (with associated colors) to be used in the classification process.

In [3]:
dm.loadCurrentProject()
classes = [ ('Class-1', "cyan"),
            ('Class-2', "green"),
            ('Class-3', "magenta"),
            ('Class-4', "blue")]

lm().setLabels( classes )


Completed Reading raster file /Users/tpmaxwel/Development/Data/Aviris/ang20190802t211646rfl/ang20190802t211646_rfl_v2p9/ang20190802t211646_corr_v2p9_img, dims = ('band', 'y', 'x'), shape = (425, 17869, 709)
#Tile[0]-> Read Data: shape = (425, 17869, 709), dims=('band', 'y', 'x')


Here we have the option to preprocess all image files (takes a long time if not done yet).

In [4]:
if preprocess:
    dm.prepare_inputs()

Here we define a custom CNN to use withing the spectraclass framework.

In [5]:
from spectraclass.learn.mlp import MLP
from spectraclass.learn.cnn import CNN2D, SpectralCNN, CNN3D

cm().nepochs = 10
cm().mid = "mlp"

mlp = MLP( 'mlp', layers = [32, 16] )
cm().addNetwork(mlp)

cnn = CNN2D( 'cnn2d', cnn_layers =  [(8,3,1)], dense_layers= [32, 16] )
cm().addNetwork(cnn)

cnn1D = SpectralCNN( 'cnn1d', cnn_layers = [(8,5,3),(8,5,3)], dense_layers= [32, 16] )
cm().addNetwork(cnn1D)

/var/folders/k6/40r5lsxs75g91q824qzfvqy40000gn/T/ipykernel_39020/265971139.py:4: UserWarning: Config option `nfeatures` not recognized by `ClassificationManager`.
  cm().nepochs = 10
2023-01-16 12:32:30.112724: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 363)]             0         
                                                                 
 dense_3 (Dense)             (None, 182)               66248     
                                                                 
 dense_4 (Dense)             (None, 91)                16653     
                                                                 
 dense_5 (Dense)             (None, 46)                4232      
                                                                 
 dense_7 (Dense)             (None, 60)                2820      
                                                                 
 dense_8 (Dense)             (None, 120)               7320      
                                                                 
 dense_9 (Dense)             (None, 240)               29040 

Here we start up the Spectraclass GUI.

In [6]:
controller: SpectraclassController = app()
controller.gui()

Initializing GUI using controller <class 'spectraclass.gui.spatial.application.Spectraclass'>
GUI Init complete
